## Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette('husl')

## Load and read data

In [ ]:
df = pd.read_csv('./data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

## Exploratory Data Analysis (EDA)

### Cleaning data

In [ ]:
df.dropna(subset='Price', inplace=True)
df.isnull().sum()

For columns **Car/Suv**, **BodyType**, the number of null is small, we consider to removing the rows with null values

In [ ]:
dropna_cols = ['Car/Suv', 'BodyType']

df.dropna(subset=dropna_cols, inplace=True)

df.isnull().sum()

#### Location

Since there are 439 null value in **Location** column, removing these rows may result in a significant loss of data. Alternatively, we can fill the null values with a `Unknown` value.

In [ ]:
df['Location'].fillna(value='Unknown', inplace=True)

#### Doors and Seats

Extract the integer value from **Doors** and **Seats** columns, fill null value with mean and round them

In [ ]:
df['Doors'] = df['Doors'].str.extract('(\d+)').astype(float)
df['Seats'] = df['Seats'].str.extract('(\d+)').astype(float)

# fillna with round mean
df['Doors'].fillna(value=round(df['Doors'].mode.iloc[0]), inplace=True)
df['Seats'].fillna(value=round(df['Seats'].mode.iloc[0]), inplace=True)

### Check null values

In [ ]:
df.isnull().sum()

# assert all values are not null
assert df.isnull().sum().sum() == 0

### Check duplicates

In [ ]:
df.duplicated().sum()

# assert no duplicates
assert df.duplicated().sum() == 0

### Convert column data type

In [ ]:
df.info()

Convert the **Kilometres**, **Price** columns to a numerical data type (float) and **Year** column to integer type.

In [ ]:
df['Kilometres'] = pd.to_numeric(df['Kilometres'], errors='coerce')
df["Price"] = pd.to_numeric(df['Price'], errors='coerce')
df['Year'] = df['Year'].astype(int)

In [ ]:
df.info()

Extract the numerical values from the **FuelConsumption**, **CylindersinEngine**, **Engine** column to convert it into a numerical data type (e.g., float). This will allow us to perform calculations or comparisons based on fuel consumption.

In [ ]:
df['FuelConsumption'] = df['FuelConsumption'].str.extract('([\d.]+) L / 100 km').astype(float)
df['CylindersinEngine'] = df['CylindersinEngine'].str.extract('(\d+)').astype(float)
df['Engine'] = df['Engine'].str.extract('([\d.]+)').astype(float)

Replace non-sense value of **Transmission** and **FuelType** with `Other`

In [ ]:
df['Transmission'] = df['Transmission'].replace('-', 'Other')
df['FuelType'] = df['FuelType'].replace('-', 'Other')

In [ ]:
df.head()

### Insight plot

#### Histograms of Numerical Features

In [ ]:
df.describe()

In [ ]:
df.hist(bins=20, figsize=(20, 10))
plt.suptitle('Histograms of Numerical Features')
plt.show()

The dataset shows diverse engine sizes with a peak around 4.42. Fuel consumption is centered around 7.67, mostly falling within the range of 6.5 to 8.8, with a few higher outliers. Kilometers driven vary widely, peaking around 100,021.81, with a right-skewed distribution indicating low-mileage for most vehicles and high-mileage outliers. The number of cylinders is mainly 4.0, with variations. Most vehicles have 4 doors, but there are outliers with 2 doors. Seat distribution centers around 5.1, with most having 5 seats, but some outliers have up to 22 seats. Vehicle prices vary widely, peaking around $37,280, with a right-skewed distribution indicating lower-priced vehicles as common but with high-priced outliers.

#### Brand Distribution

In [ ]:
plt.figure(figsize=(20, 6))
brand_counts = df['Brand'].value_counts()
plt.bar(brand_counts.index, brand_counts)
plt.title('Brand Distribution')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.xticks(rotation=90, ha='right')
plt.show()

Figure shows that `Toyota` is the most popular brand in AUS with over 2500 car models, twice as many as the second brand Hyundai. The third is Mazada has little different from `Hyundai`.

`Holden`, `Ford`, `Mitsubishi`, `Nissan` has no much difference from number of car model.

#### Transmission Type Distribution

In [ ]:
transmission_counts = df['Transmission'].value_counts()
plt.bar(transmission_counts.index, transmission_counts)
plt.title('Transmission Type Distribution')
plt.xlabel('Transmission Type')
plt.ylabel('Count')
plt.show()

The number of car models using automatic transmission is the largest among the 3 types of transmissions

#### Fuel Type Distribution

In [ ]:
plt.figure(figsize=(20, 6))

fuel_counts = df['FuelType'].value_counts()
plt.bar(fuel_counts.index, fuel_counts)
plt.title('Fuel Type Distribution')
plt.xlabel('Fuel Type')
plt.ylabel('Count')
plt.show()

Car models using `Unleaded`, `Diesel`, `Premium` has the highest proportion

#### Used or New Car distribution

In [ ]:
used_vs_new_counts = df['UsedOrNew'].value_counts()
plt.bar(used_vs_new_counts.index, used_vs_new_counts)
plt.title('Used vs. New Cars')
plt.xlabel('Used or New')
plt.ylabel('Count')
plt.show()

The category with the highest model count is used cars, followed by new cars. Demo cars have a comparatively smaller representation.

#### BodyType distribution

In [ ]:
plt.figure(figsize=(20, 6))
brand_counts = df['BodyType'].value_counts()
plt.bar(brand_counts.index, brand_counts)
plt.title('Body Type Distribution')
plt.xlabel('Body Type')
plt.ylabel('Count')
plt.xticks(rotation=90, ha='right')
plt.show()

The `SUV` category leads with almost 7000 models, followed by other popular types such as `Hatchback`, `Ute/Tray`, `Sedan`, and `Wagon`.

## Questions

### 1. What is the distribution of vehicle age (based on the year column) for new versus used vehicles?

In [ ]:
current_year = datetime.now().year
df['VehicleAge'] = current_year - df['Year']

# Separate the dataset into new and used vehicles
new_vehicles = df[df['UsedOrNew'].str.lower() == 'new']
used_vehicles = df[df['UsedOrNew'].str.lower() == 'used']

# Plot the distribution of vehicle age for new and used vehicles
plt.figure(figsize=(12, 6))

# Plot for new vehicles
plt.subplot(2, 1, 1)
new_vehicles['VehicleAge'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of Vehicle Age for New Vehicles')
plt.xlabel('Vehicle Age (years)')
plt.ylabel('Frequency')

# Plot for used vehicles
plt.subplot(2, 1, 2)
used_vehicles['VehicleAge'].value_counts().sort_index().plot(kind='bar', color='orange')
plt.title('Distribution of Vehicle Age for Used Vehicles')
plt.xlabel('Vehicle Age (years)')
plt.ylabel('Frequency')

# Show the plots
plt.tight_layout()
plt.show()

#### **Analyze the results**:

**1. New vehicles:**
- The distribution of new vehicles is heavily skewed towards more recent years, with a concentration in the years 2023, 2022.

- This skewness suggests that consumers are predominantly purchasing newer models, potentially driven by factors such as advanced features, improved technology, or warranty coverage.

**2. Used vehicles:**
- The distribution of used vehicles is more evenly spread across a wider range of manufacturing years, including both recent and older models. Accordingly, it focused on the old vehicles which are around 5, 6, 7 year old.

- This broader distribution indicates a diverse selection of used vehicles in the market, providing consumers with options across various age groups.

**3. Market Dynamics:**
- The analysis highlights a clear distinction in the market dynamics between new and used vehicles. New vehicles showcase a trend of rapid turnover, while used vehicles offer a more diverse range of options, accommodating a broader budget spectrum and catering to individuals seeking different model years.

**4. Consumer Preferences:**
- Consumer preferences for new vehicles appear to align with the desire for the latest technologies and designs, as evidenced by the concentration in recent manufacturing years.

- In contrast, the used vehicle market caters to consumers with varied preferences, including those looking for cost-effective options, classic models, or specific features that may not be present in the newest releases

**5. Market Opportunities:**
- Businesses in the automotive industry can capitalize on the observed trends. Dealerships may focus marketing efforts on promoting the latest models to appeal to consumers interested in new vehicles. Simultaneously, there is an opportunity to emphasize the diverse selection of used vehicles, targeting a broader audience.

### 2. How does the fuel consumption vary across different car brands (top 10 popular brands)?

Fuel Consumption Across top 10 popular brands

In [ ]:
top_10_brands_list = df['Brand'].value_counts().head(10).index
filtered_df = df[df['Brand'].isin(top_10_brands_list)][['FuelConsumption', 'Brand']]

In [ ]:
filtered_df.groupby('Brand')['FuelConsumption'].describe()

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(x='Brand', y='FuelConsumption', data=filtered_df)
plt.title('Fuel Consumption Across Different Car Brands')
plt.xlabel('Car Brand')
plt.ylabel('Fuel Consumption (L/100km)')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.show()

Which brand offers the most fuel-efficient vehicles on average?

In [ ]:
# Calculate the average fuel consumption for each brand
avg_fuel_efficiency = filtered_df.groupby('Brand')['FuelConsumption'].mean().reset_index()

# Sort the results to find the brand with the lowest average fuel consumption
avg_fuel_efficiency = avg_fuel_efficiency.sort_values('FuelConsumption', ascending=True)

# Display the results
print(avg_fuel_efficiency)

# Visualize the results using a bar chart
plt.figure(figsize=(20, 15))
sns.barplot(x='FuelConsumption', y='Brand', data=avg_fuel_efficiency, color='teal')
plt.title('Average Fuel Consumption by Brand')
plt.xlabel('Average Fuel Consumption (L/100km)')
plt.ylabel('Brand')
plt.show()

#### **Analyze the result:**

**1. Observation:**
- Toyota has the highest count, indicating a large representation in the dataset.

- Holden has the highest average fuel consumption among the listed brands.

- Volkswagen has the lowest average fuel consumption, suggesting higher fuel efficiency on average.

- There is variability in fuel consumption within each brand, as evident from the standard deviation values.

- The minimum fuel consumption for all brands is 0.0, which may indicate missing or erroneous data points.

- Brands like Mercedes-Benz and Volkswagen have relatively higher variability in fuel consumption, as indicated by their higher standard deviations.

**2. Market Opportunities:**

- Target Efficiency: Brands with lower fuel consumption (e.g., Volkswagen) can target environmentally conscious and cost-conscious consumers.

- Innovation for High Consumers: Brands with higher fuel consumption (e.g., Holden) can invest in research for fuel-efficient technologies, including hybrid or electric options.

- Diversify Offerings: Brands with a wide range of fuel consumption (e.g., Ford, Toyota) have opportunities to diversify products, appealing to various consumer preferences.

- Leverage Brand Image: Higher-end brands (e.g., Mercedes-Benz) can capitalize on luxury and innovation, aligning with consumer preferences beyond fuel efficiency.

- Educate Consumers: Brands with variability (e.g., Mercedes-Benz) can run educational campaigns to inform consumers about the range of fuel-efficient options within the brand.

### 3. Which location has the highest average price for cars?

Extract state in location

In [ ]:
def extract_state(value):
    if ',' in value:
        res = value.split(',')[1].strip()
        if res == 'AU-VIC':
            return 'VIC'
        return res
    return value

df['State'] = df['Location'].apply(extract_state)
df['State'].unique()

Visualize

In [ ]:
# Group the data by location and calculate the average price for each location
average_price_by_location = df.groupby('State')['Price'].mean().sort_values(ascending=False)

# Plot the results
plt.figure(figsize=(20, 10))
bar_plot = average_price_by_location.plot(kind='bar', color='skyblue')
plt.title('Average Car Price by Location')
plt.xlabel('State')
plt.ylabel('Average Car Price')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability

# Add data labels to the bars
for index, value in enumerate(average_price_by_location):
    bar_plot.text(index, value, f'${value:.2f}', ha='center', va='bottom')

plt.show()

#### **Analyze the result:**

Here are some of the Market Dynamics and Opportunities:

**1. Identifying High-Value Markets**:
- "Unknown" locations have the highest average car prices, significantly surpassing prices in recognized states and territories.

- Businesses can explore and understand the factors contributing to high prices in these unknown locations, potentially uncovering lucrative, niche markets with higher consumer purchasing power.

**2. Targeting Affluent Regions**:
- States like New South Wales (NSW) and Victoria (VIC) have relatively high average car prices.

- There is an opportunity for dealerships and manufacturers to target these affluent regions with premium or luxury vehicle offerings, as consumers in these 
areas may be willing to spend more on high-end models.

**3. Capitalizing on Demand in ACT**:
- The Australian Capital Territory (ACT) has a notable average car price, indicating a potentially affluent market.

- Businesses can tailor marketing strategies to emphasize premium features, advanced technologies, or exclusive models to appeal to consumers in the ACT.

**4. Understanding Regional Preferences**:
- Tasmania (TAS) and Northern Territory (NT) exhibit higher average prices compared to other states.

- Analyzing local preferences, such as a preference for specific vehicle types or features, can help businesses tailor their inventory to meet the demands of these regions.

**5. Exploring Growth in Western Australia (WA)**:
- Western Australia (WA) has a competitive average car price.

- Businesses might explore opportunities to further tap into this market by understanding local preferences and potentially introducing models that align with the lifestyle and preferences of consumers in WA.

**6. Offering Value in Queensland (QLD) and South Australia (SA)**:

- Queensland (QLD) and South Australia (SA) have relatively lower average car prices.

- Opportunities lie in offering value-oriented vehicles, promotions, or financing options to appeal to a broader consumer base in these regions.

**7. Adapting Strategies for Unknown Locations**:

- Further investigation is needed for the "Unknown" category with the highest average car prices.

- Businesses may want to uncover and target specific cities or regions within this category, adapting their strategies to capitalize on the high-price market potential.


###  4. Is there a correlation between numerical data ?

In [ ]:
numerical_features = df.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numerical_features.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

#### **Analyze the results:**

**1. Analyze**:

*1.1. Year and VehicleAge*

- There is a perfect negative correlation of -1.000 between "Year" and "VehicleAge," which is expected. As the manufacturing year decreases, the age of the vehicle increases.

*1.2. Engine and CylindersinEngine:*

- There is a positive correlation of 1.000 between "Engine" and "CylindersinEngine," indicating a strong linear relationship. This is expected, as the number of cylinders in the engine is directly related to its size (displacement).

*1.3. Kilometres and VehicleAge:*

- There is a strong positive correlation of 0.728 between "Kilometres" and "VehicleAge." This suggests that older vehicles tend to have higher mileage, which is a logical relationship.

*1.4. Doors and Seats:*

- There is a moderate positive correlation of 0.357 between "Doors" and "Seats." This indicates that vehicles with more doors tend to have more seats, which aligns with typical car configurations.

*1.5. Price and Year, Engine, Kilometres, CylindersinEngine:*

- "Price" shows positive correlations with "Year" (0.361), "Engine" (0.326), and "CylindersinEngine" (0.326). This suggests that newer vehicles, larger engine sizes, and more cylinders are associated with higher prices.

- There is a negative correlation of -0.398 between "Price" and "Kilometres," indicating that as the mileage increases, the price tends to decrease.

*1.6. FuelConsumption and other Variables:*

- "FuelConsumption" shows weak correlations with other variables, indicating that fuel consumption is not strongly linked to variables such as year, engine size, or kilometers.

**2. Observation:**

- Newer vehicles tend to be more expensive.

- Engine size and the number of cylinders are positively correlated.

- Older vehicles have higher mileage.

- More doors are associated with more seats.

- Price is influenced by factors like newer manufacturing year, larger engine size, and fewer kilometers.

**3.Market Opportunities:**

- Dealerships can emphasize the value of newer models, larger engines, and lower mileage in marketing campaigns to potentially justify higher prices.

- Manufacturers could explore producing vehicles with larger engines, aligning with consumer preferences that contribute to higher prices.

- There may be opportunities to market older vehicles with higher mileage as budget-friendly options.


### 5. What is the average price difference between new and used cars for each brand?

In [ ]:
# Group the data by brand and new/used category, and calculate the average price for each group
average_price_by_brand_category = df[df['Brand'].isin(top_10_brands_list)].groupby(['Brand', 'UsedOrNew'])['Price'].mean().reset_index()

# Create a bar plot to visualize the average price difference between new and used cars for each brand
plt.figure(figsize=(15, 8))
sns.barplot(x='Brand', y='Price', hue='UsedOrNew', data=average_price_by_brand_category)
plt.title('Average Price Difference Between New and Used Cars for Each Brand')
plt.xlabel('Car Brand')
plt.ylabel('Average Price')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.legend(title='New or Used')
plt.show()

#### **Analyze the results:**

**1. Analyze:**

*1.1. Brand Variations:*

- Each brand is represented across three categories: "DEMO," "NEW," and "USED."

- Brands like Ford, Hyundai, Kia, Mazda, Mercedes-Benz, Mitsubishi, Nissan, Toyota, and Volkswagen are included in the dataset.

*1.2. Price Differences:*

- The "Price" column represents the average price for each category and brand.

- The "DEMO" category typically has prices higher than the "NEW" category for brands like Ford, Hyundai, Kia, Mazda, Mercedes-Benz, Mitsubishi, Nissan, and Volkswagen. This may be due to demo vehicles often having additional features or being more equipped for test drives.
- The "USED" category generally has lower prices compared to both "DEMO" and "NEW" categories for most brands. This is expected, as used cars usually have lower prices than their new counterparts.

*1.3. Brand-Specific Observations:*

- Mercedes-Benz has a significant price difference between new and used cars, with new cars being substantially more expensive than used ones.

- Some brands, such as Ford, Hyundai, and Kia, have higher average prices for demo cars compared to new cars. This might be due to specific features or conditions of demo vehicles.

- Toyota shows a smaller price difference between new and used cars compared to some other brands.

**2. Market Dynamics:**

- Brands with higher average prices for new cars may be positioned as offering premium or high-end models.

- Brands with smaller price differences or lower prices for new cars may attract cost-conscious consumers looking for more budget-friendly options.

**3. Market Opportunities:**

- Dealerships can capitalize on the popularity of demo cars by highlighting their features and offering test drives to potential customers.

- Brands with significant price differences between new and used cars may explore marketing strategies to emphasize the value, features, or warranties associated with new models.

- Brands with smaller price differences or lower average prices for new cars can target a broader consumer base by appealing to budget-conscious buyers.

### 6. What types of vehicles (brand, body type) are more popular in different locations?

In [ ]:
# Group by 'Location', 'Brand', and 'BodyType', then count the occurrences
popularity = df.groupby(['State', 'Brand', 'BodyType']).size().reset_index(name='Counts')

# Sort the results to find the most popular combinations
popularity = popularity.sort_values(by=['State', 'Counts'], ascending=[True, False])

# Since there could be many locations, let's visualize the top N popular vehicle types for the first few locations
top_n = 20
unique_locations = popularity['State'].unique()

# Create a single figure with subplots
fig, axes = plt.subplots(nrows=len(unique_locations), figsize=(10, 6 * len(unique_locations)))

for i, location in enumerate(unique_locations):
    location_data = popularity[popularity['State'] == location].head(top_n)
    
    # Use the current subplot for each location
    ax = axes[i]
    
    sns.barplot(x='Counts', y='Brand', hue='BodyType', data=location_data, dodge=False, ax=ax)
    ax.set_title(f'Most Popular Vehicle Types in {location}')
    ax.set_xlabel('Number of Vehicles')
    ax.set_ylabel('Brand')
    ax.legend(title='Body Type')

# Adjust layout to prevent overlapping
plt.tight_layout()
plt.show()

#### **Analyze the results:**

**1. Observations:**

*1.1. SUV Dominance:*

- SUVs are consistently popular across all states, representing a significant portion of the vehicle market. Brands like Toyota, Mazda, Hyundai, and Mitsubishi often dominate the SUV category.

*1.2. Toyota's Overall Dominance:*

- Toyota stands out as the most popular brand in almost every state, with a strong presence across various body types, including SUVs, Ute/Tray, Hatchback, Sedan, and Wagon.

*1.3. Body Type Variations:*

- While SUVs are dominant, there is diversity in body types across states. Ute/Tray, Hatchback, Sedan, Wagon, and Commercial vehicles are present in varying degrees, reflecting diverse consumer preferences.

*1.4. Regional Preferences:*

- Each state shows unique preferences for specific brands and body types. For instance, Mitsubishi is popular in SA, Mazda in VIC, and Nissan in QLD, showcasing regional variations in brand preferences.

*1.5. Premium Brands in Certain States:*

- Premium brands like BMW, Audi, Mercedes-Benz, and Lexus are more prevalent in certain states, reflecting potential differences in consumer affluence and preferences.

*1.6. Commercial Vehicles:*

- Commercial vehicles, including Ute/Tray and Commercial categories, are present across states, highlighting the importance of these vehicles for businesses and trade.

**2. Market Opportunities:**

- **Capitalizing on SUV Popularity**: Manufacturers and dealerships can capitalize on the continued popularity of SUVs by introducing new models, features, or variants that align with consumer preferences.

- **Strategic Brand Positioning**: Brands can strategically position themselves based on regional preferences. Understanding which brands are popular in specific states allows for more targeted marketing and product development.

- **Diversification of Body Types**: Offering a diverse range of body types, including Ute/Tray, Hatchback, Sedan, and Wagon, can help car manufacturers cater to a broader audience with varying needs and preferences.

- **Competitive Analysis**: Brands can conduct competitive analyses to understand their market share in each state and identify opportunities for growth or improvement.

- **Premium Brand Presence**: Premium brands should focus on strengthening their presence in states where they are less dominant, potentially through strategic marketing campaigns or expanding their model offerings.